In [1]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "Informer-Tensorflow/models")
sys.path.insert(1, "Informer-Tensorflow")
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

2023-07-07 19:42:55.805138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from attn import ProbAttention, AttentionLayer

In [3]:
class MultiHeadProbAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadProbAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.attn = AttentionLayer(ProbAttention(False), self.d_model, self.num_heads)
    def call(self, inputs):
        query = inputs
        key = inputs
        value = inputs
        x = self.attn([query, key, value])
        return x


In [4]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    # def compute_mask(self, *args, **kwargs):
    #     return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        length = tf.shape(x)[1]
        # x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [5]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, c_in):
        super(ConvLayer, self).__init__()
        self.downConv = tf.keras.layers.Conv1D(
                                  filters=c_in,
                                  kernel_size=3,
                                  padding='causal')
        self.norm = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.ELU()
        self.maxPool = tf.keras.layers.MaxPool1D(pool_size=3)#, strides=2)

    def call(self, x, **kargs):
        x = self.downConv(x)
        x = self.norm(x)
        x = self.activation(x)
        x = self.maxPool(x)
        return x

In [6]:
def models():
    inputs = keras.layers.Input(shape=(2000,1), batch_size=6)
    # x = keras.layers.Dense(32, activation="tanh")(inputs)
    x = keras.layers.Conv1D(32, 1, activation='tanh')(inputs)
    x = PositionalEmbedding(32)(x)
    x = MultiHeadProbAttention(128, 32)(x)
    x = ConvLayer(64)(x)
    x = MultiHeadProbAttention(64, 32)(x)
    x = ConvLayer(32)(x)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(x)
    x = keras.layers.Dense(5)(x)
    return keras.Model(inputs, x)

In [7]:
model = models()
model.summary()
# del model

2023-07-07 19:42:58.058205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 19:42:58.091198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 19:42:58.093016: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[TensorShape([6, 2000, 32]), TensorShape([6, 2000, 32]), TensorShape([6, 2000, 32])]
[TensorShape([6, 666, 64]), TensorShape([6, 666, 64]), TensorShape([6, 666, 64])]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(6, 2000, 1)]            0         
                                                                 
 conv1d (Conv1D)             (6, 2000, 32)             64        
                                                                 
 positional_embedding (Posit  (6, 2000, 32)            0         
 ionalEmbedding)                                                 
                                                                 
 multi_head_prob_attention (  (6, 2000, 128)           29184     
 MultiHeadProbAttention)                                         
                                                                 
 conv_layer (ConvLayer)   

In [8]:
x = tf.random.normal((6, 2000, 1))
model(x)

2023-07-07 19:43:02.111591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-07 19:43:02.625890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


<tf.Tensor: shape=(6, 5), dtype=float32, numpy=
array([[-3.3742445e-08, -8.4900691e-08,  3.4490128e-08,  6.0140628e-08,
         1.7478055e-08],
       [ 7.4714727e-08,  3.7547728e-08, -4.7708657e-08,  1.0721498e-08,
        -4.0473108e-08],
       [-2.5145393e-08, -5.1822688e-08,  1.0447266e-08,  6.5085164e-09,
        -8.0555665e-08],
       [-2.0503242e-08, -3.6908485e-09,  4.2766153e-08,  5.3343303e-09,
        -5.7435436e-08],
       [-1.9883842e-08, -4.0068631e-08,  4.9384013e-08, -9.9404687e-08,
        -8.6990518e-08],
       [ 1.0486993e-07,  7.9462609e-08, -1.8148214e-08, -4.2659586e-08,
        -4.5853898e-08]], dtype=float32)>

In [9]:
data = np.load("data_shuffled.npz")
X = data["X"]
Y = data["Y"]



In [10]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [11]:
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.experimental.AdamW(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=["accuracy"])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./model_checkpoints', monitor='val_accuracy', save_best_only=True)


In [15]:


model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])


In [17]:
model.fit(x=X[0:846], y=Y[0:846], batch_size=6, epochs=1000, validation_data=(X[846:996], Y[846:996]),
         callbacks=[tensorboard_callback])


Epoch 1/1000
141/141 [==============================] - 11s 76ms/step - loss: 1.5733 - accuracy: 0.4243 - val_loss: 1.5404 - val_accuracy: 0.3200
Epoch 2/1000
141/141 [==============================] - 11s 75ms/step - loss: 1.4505 - accuracy: 0.4113 - val_loss: 1.3404 - val_accuracy: 0.4200
Epoch 3/1000
141/141 [==============================] - 12s 86ms/step - loss: 1.2561 - accuracy: 0.4066 - val_loss: 1.1640 - val_accuracy: 0.4267
Epoch 4/1000
141/141 [==============================] - 11s 76ms/step - loss: 1.1727 - accuracy: 0.3830 - val_loss: 1.1279 - val_accuracy: 0.4267
Epoch 5/1000
141/141 [==============================] - 11s 78ms/step - loss: 1.1394 - accuracy: 0.4161 - val_loss: 1.1089 - val_accuracy: 0.4333
Epoch 6/1000
141/141 [==============================] - 11s 76ms/step - loss: 1.1453 - accuracy: 0.4031 - val_loss: 1.1031 - val_accuracy: 0.4333
Epoch 7/1000
141/141 [==============================] - 11s 76ms/step - loss: 1.1643 - accuracy: 0.4196 - val_loss: 1.1038 -

Epoch 57/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.3402 - accuracy: 0.8818 - val_loss: 0.6625 - val_accuracy: 0.8133
Epoch 58/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.3203 - accuracy: 0.8972 - val_loss: 0.4664 - val_accuracy: 0.8533
Epoch 59/1000
141/141 [==============================] - 12s 87ms/step - loss: 0.2743 - accuracy: 0.9113 - val_loss: 1.7108 - val_accuracy: 0.4933
Epoch 60/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.2733 - accuracy: 0.9173 - val_loss: 0.7692 - val_accuracy: 0.7133
Epoch 61/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.2803 - accuracy: 0.9125 - val_loss: 0.4157 - val_accuracy: 0.8733
Epoch 62/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.2593 - accuracy: 0.9184 - val_loss: 0.8455 - val_accuracy: 0.6867
Epoch 63/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.2633 - accuracy: 0.9113 - val_loss: 0

141/141 [==============================] - 11s 76ms/step - loss: 0.1291 - accuracy: 0.9563 - val_loss: 0.6136 - val_accuracy: 0.8533
Epoch 113/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1156 - accuracy: 0.9598 - val_loss: 2.3501 - val_accuracy: 0.5800
Epoch 114/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1253 - accuracy: 0.9504 - val_loss: 3.1393 - val_accuracy: 0.2533
Epoch 115/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1396 - accuracy: 0.9563 - val_loss: 0.3826 - val_accuracy: 0.8733
Epoch 116/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1425 - accuracy: 0.9492 - val_loss: 18.3733 - val_accuracy: 0.2000
Epoch 117/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1129 - accuracy: 0.9598 - val_loss: 0.4408 - val_accuracy: 0.8800
Epoch 118/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.1283 - accuracy: 0.9551 - val_loss: 5.5774 -

141/141 [==============================] - 11s 76ms/step - loss: 0.0717 - accuracy: 0.9752 - val_loss: 11.9647 - val_accuracy: 0.2267
Epoch 168/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0754 - accuracy: 0.9728 - val_loss: 1.5676 - val_accuracy: 0.5733
Epoch 169/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0686 - accuracy: 0.9728 - val_loss: 0.4259 - val_accuracy: 0.9000
Epoch 170/1000
141/141 [==============================] - 12s 88ms/step - loss: 0.0846 - accuracy: 0.9740 - val_loss: 1.5643 - val_accuracy: 0.6267
Epoch 171/1000
141/141 [==============================] - 12s 79ms/step - loss: 0.1113 - accuracy: 0.9657 - val_loss: 0.3662 - val_accuracy: 0.8933
Epoch 172/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0782 - accuracy: 0.9704 - val_loss: 0.5766 - val_accuracy: 0.8600
Epoch 173/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0825 - accuracy: 0.9728 - val_loss: 8.8404 -

141/141 [==============================] - 11s 76ms/step - loss: 0.0540 - accuracy: 0.9835 - val_loss: 0.4399 - val_accuracy: 0.9133
Epoch 223/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0574 - accuracy: 0.9799 - val_loss: 0.3987 - val_accuracy: 0.9067
Epoch 224/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0407 - accuracy: 0.9870 - val_loss: 0.5802 - val_accuracy: 0.8800
Epoch 225/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0466 - accuracy: 0.9799 - val_loss: 1.9600 - val_accuracy: 0.6800
Epoch 226/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0422 - accuracy: 0.9882 - val_loss: 1.1975 - val_accuracy: 0.7000
Epoch 227/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0753 - accuracy: 0.9740 - val_loss: 0.7607 - val_accuracy: 0.7733
Epoch 228/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0360 - accuracy: 0.9870 - val_loss: 0.7136 - 

...

141/141 [==============================] - 11s 77ms/step - loss: 0.0123 - accuracy: 0.9965 - val_loss: 0.5190 - val_accuracy: 0.9067
Epoch 773/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0115 - accuracy: 0.9976 - val_loss: 0.6851 - val_accuracy: 0.9000
Epoch 774/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0236 - accuracy: 0.9941 - val_loss: 0.5447 - val_accuracy: 0.9133
Epoch 775/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0157 - accuracy: 0.9953 - val_loss: 0.6044 - val_accuracy: 0.8933
Epoch 776/1000
141/141 [==============================] - 11s 78ms/step - loss: 0.0160 - accuracy: 0.9965 - val_loss: 0.5022 - val_accuracy: 0.9333
Epoch 777/1000
141/141 [==============================] - 11s 78ms/step - loss: 0.0256 - accuracy: 0.9929 - val_loss: 1.7960 - val_accuracy: 0.5733
Epoch 778/1000
141/141 [==============================] - 11s 78ms/step - loss: 0.0068 - accuracy: 0.9988 - val_loss: 0.7556 - 

141/141 [==============================] - 11s 76ms/step - loss: 0.0132 - accuracy: 0.9941 - val_loss: 1.9489 - val_accuracy: 0.5200
Epoch 828/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0066 - accuracy: 0.9988 - val_loss: 0.6966 - val_accuracy: 0.8933
Epoch 829/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0169 - accuracy: 0.9929 - val_loss: 1.1904 - val_accuracy: 0.8000
Epoch 830/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0070 - accuracy: 0.9988 - val_loss: 1.1728 - val_accuracy: 0.7733
Epoch 831/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0091 - accuracy: 0.9953 - val_loss: 2.1231 - val_accuracy: 0.6467
Epoch 832/1000
141/141 [==============================] - 11s 78ms/step - loss: 0.0110 - accuracy: 0.9953 - val_loss: 1.0092 - val_accuracy: 0.8267
Epoch 833/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0084 - accuracy: 0.9976 - val_loss: 0.6721 - 

141/141 [==============================] - 11s 79ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.6348 - val_accuracy: 0.9267
Epoch 883/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0042 - accuracy: 0.9988 - val_loss: 0.7710 - val_accuracy: 0.8667
Epoch 884/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0132 - accuracy: 0.9941 - val_loss: 0.5161 - val_accuracy: 0.9400
Epoch 885/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0076 - accuracy: 0.9953 - val_loss: 0.5509 - val_accuracy: 0.9200
Epoch 886/1000
141/141 [==============================] - 12s 89ms/step - loss: 0.0071 - accuracy: 0.9988 - val_loss: 0.6401 - val_accuracy: 0.9267
Epoch 887/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0145 - accuracy: 0.9953 - val_loss: 0.6326 - val_accuracy: 0.9267
Epoch 888/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0097 - accuracy: 0.9965 - val_loss: 1.3179 - 

141/141 [==============================] - 11s 76ms/step - loss: 0.0181 - accuracy: 0.9965 - val_loss: 0.5250 - val_accuracy: 0.9267
Epoch 938/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0055 - accuracy: 0.9988 - val_loss: 0.7422 - val_accuracy: 0.9067
Epoch 939/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0067 - accuracy: 0.9988 - val_loss: 0.5821 - val_accuracy: 0.9133
Epoch 940/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.7086 - val_accuracy: 0.9267
Epoch 941/1000
141/141 [==============================] - 12s 87ms/step - loss: 0.0143 - accuracy: 0.9953 - val_loss: 0.5317 - val_accuracy: 0.9267
Epoch 942/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0051 - accuracy: 0.9988 - val_loss: 0.7332 - val_accuracy: 0.9133
Epoch 943/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0136 - accuracy: 0.9965 - val_loss: 0.5557 - 

141/141 [==============================] - 11s 77ms/step - loss: 0.0096 - accuracy: 0.9965 - val_loss: 0.7501 - val_accuracy: 0.8867
Epoch 993/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 0.5255 - val_accuracy: 0.9267
Epoch 994/1000
141/141 [==============================] - 11s 76ms/step - loss: 0.0104 - accuracy: 0.9941 - val_loss: 0.9496 - val_accuracy: 0.8467
Epoch 995/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 0.8388 - val_accuracy: 0.8800
Epoch 996/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0120 - accuracy: 0.9953 - val_loss: 0.8723 - val_accuracy: 0.8867
Epoch 997/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0142 - accuracy: 0.9941 - val_loss: 0.9741 - val_accuracy: 0.8133
Epoch 998/1000
141/141 [==============================] - 11s 77ms/step - loss: 0.0114 - accuracy: 0.9976 - val_loss: 0.5850 - 